# Digit Recognizer

Description of kaggle competition.

## 1. Import Modules

In [62]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np
import pandas as pd

## 2. Data

### 2.1. Preamble/Parameters

In [37]:
# Input image dimensions
img_rows, img_cols = 28, 28

# Number of classes into which we shall categorise our examples.
num_classes = 10

### 2.2. Load Data 

Load the MNIST data set keras datsets (already split into training and test data set).

In [38]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2.3 Pre-Process Data

First we reshape the data.

In [39]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

Then we convert the data from integer to floating point.

In [40]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

Rescale data values from (0,255) to (0,1) to speed up convergence. 

In [41]:
x_train /= 255
x_test /= 255

Display information about data to confirm dimensions.

In [42]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Convert class vectors to binary class matrices (map values in 0,1,...,9 to unit vectors in R^10).

In [43]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## 3. Build Convolutional Neural Net

### 3.1. Define Net Structure

Use keras Sequential module to build net layer by layer.

In [44]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### 3.2 Train

Set the batch size and number of epochs to be used in training the net.

In [45]:
batch_size = 128
epochs = 12

Fit the model to the training data.

In [46]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 70s 1ms/step - loss: 0.2640 - accuracy: 0.9177 - val_loss: 0.0552 - val_accuracy: 0.9826
Epoch 2/12
60000/60000 [==============================] - 71s 1ms/step - loss: 0.0855 - accuracy: 0.9749 - val_loss: 0.0435 - val_accuracy: 0.9852
Epoch 3/12
60000/60000 [==============================] - 74s 1ms/step - loss: 0.0633 - accuracy: 0.9814 - val_loss: 0.0359 - val_accuracy: 0.9874
Epoch 4/12
60000/60000 [==============================] - 71s 1ms/step - loss: 0.0514 - accuracy: 0.9843 - val_loss: 0.0319 - val_accuracy: 0.9894
Epoch 5/12
60000/60000 [==============================] - 72s 1ms/step - loss: 0.0445 - accuracy: 0.9864 - val_loss: 0.0287 - val_accuracy: 0.9907
Epoch 6/12
60000/60000 [==============================] - 69s 1ms/step - loss: 0.0390 - accuracy: 0.9882 - val_loss: 0.0267 - val_accuracy: 0.9922
Epoch 7/12
60000/60000 [==============================] - 81s 1ms/st

### 3.3 Save Model to File

Save model to JSON file and weights to HDF5 file. 

In [78]:
# Serialize model to JSON.
model_json = model.to_json()
with open('./models/model_keras_cnn_mnist_default.json', 'w') as json_file:
    json_file.write(model_json)
    
# Serialize weights to HDF5.
model.save_weights('./weights/model_keras_cnn_mnist_default.h5')

# Output message.
print('Saved model to disk.')

Saved model to disk


To load model at a later date...

In [79]:
# # Load json and create model.
# json_file = open('./models/model_keras_cnn_mnist_default.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)

# # Load weights into new model.
# loaded_model.load_weights('model.h5')

# print('Loaded model from disk.')

## kaggle Data

In [59]:
# Load the data
ktest = pd.read_csv("./data/test.csv")

# Reshape the data.
ktest = ktest.values.reshape(-1,28,28,1)

In [60]:
ktest.shape

(28000, 28, 28, 1)

In [72]:
# Make predictions using model.
predictions = model.predict(ktest)

In [73]:
# Convert binary matrices into labels.
predictions = np.argmax(predictions,axis=1)

In [74]:
# Add label to column (ready for submission).
predictions = pd.Series(predictions,name='Label')

In [75]:
# Add ImageID column (for submission).
predictions = pd.concat([pd.Series(range(1,28001),name='ImageId'),predictions], axis=1)

In [76]:
# Save submission as .csv file.
predictions.to_csv('./submissions/keras_cnn_mnist_default.csv',index=False,header=True)